# 🎨 Python Abstraction Masterclass

## 📚 Table of Contents
1. Understanding Abstraction
2. Basic Abstraction Techniques
3. Abstract Base Classes
4. Interface Design
5. Design Patterns with Abstraction
6. Real-World Applications
7. Advanced Abstraction Concepts
8. Enterprise-Level Examples

## 🎯 Learning Objectives
After completing this notebook, you will:
- Master the concept of abstraction
- Implement abstract classes and methods
- Design clean interfaces
- Apply abstraction in real-world scenarios
- Use advanced abstraction patterns

## 1. Understanding Abstraction 🎨

Abstraction is hiding complex implementation details and showing only the necessary features of an object.

```
    📱 Smartphone Interface
    ┌─────────────────────┐
    │     Make Call       │
    │     Send Text       │  User sees these
    │     Take Photo      │  simple operations
    └─────────────────────┘
            |
    ┌─────────────────────┐
    │ Signal Processing   │
    │ Data Encryption     │  Complex internals
    │ Image Processing    │  are hidden
    └─────────────────────┘
```

In [ ]:
# Basic Abstraction Example
class Smartphone:
    def __init__(self):
        self.__battery_level = 100
        self.__signal_strength = 0
    
    def make_call(self, number):
        if self.__check_battery() and self.__check_signal():
            print(f"Calling {number}...")
            self.__use_battery(5)
        else:
            print("Cannot make call")
    
    def __check_battery(self):
        return self.__battery_level > 5
    
    def __check_signal(self):
        self.__signal_strength = self.__get_network_signal()
        return self.__signal_strength > 2
    
    def __get_network_signal(self):
        # Complex signal processing logic
        return 4
    
    def __use_battery(self, amount):
        self.__battery_level -= amount

# Using the smartphone
phone = Smartphone()
phone.make_call("123-456-7890")  # Simple interface for complex operation

## 2. Basic Abstraction Techniques 🛠️

### 2.1 Method Abstraction

In [ ]:
class CoffeeMachine:
    def __init__(self):
        self.__water_level = 100
        self.__coffee_beans = 100
        self.__milk = 100
    
    def make_coffee(self, coffee_type):
        """Public interface for making coffee"""
        if not self.__check_resources():
            return "Please refill resources"
        
        if coffee_type == "espresso":
            return self.__make_espresso()
        elif coffee_type == "latte":
            return self.__make_latte()
        else:
            return "Unknown coffee type"
    
    def __check_resources(self):
        return all([self.__water_level > 20,
                   self.__coffee_beans > 10,
                   self.__milk > 0])
    
    def __make_espresso(self):
        self.__water_level -= 20
        self.__coffee_beans -= 10
        return "Your espresso is ready!"
    
    def __make_latte(self):
        self.__water_level -= 20
        self.__coffee_beans -= 10
        self.__milk -= 30
        return "Your latte is ready!"

# Using the coffee machine
machine = CoffeeMachine()
print(machine.make_coffee("latte"))  # Simple interface hides complexity

## 3. Abstract Base Classes 🏗️

### 3.1 Basic Abstract Class Example

In [ ]:
from abc import ABC, abstractmethod

class PaymentProcessor(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass
    
    @abstractmethod
    def refund_payment(self, amount):
        pass

class CreditCardProcessor(PaymentProcessor):
    def __init__(self, api_key):
        self.__api_key = api_key
        self.__gateway = self.__connect_to_gateway()
    
    def process_payment(self, amount):
        # Complex payment processing logic
        return f"Processing ${amount} via Credit Card"
    
    def refund_payment(self, amount):
        return f"Refunding ${amount} to Credit Card"
    
    def __connect_to_gateway(self):
        # Complex connection logic
        return "Connected to payment gateway"

class PayPalProcessor(PaymentProcessor):
    def process_payment(self, amount):
        return f"Processing ${amount} via PayPal"
    
    def refund_payment(self, amount):
        return f"Refunding ${amount} to PayPal"

# Using payment processors
cc_processor = CreditCardProcessor("api_key_123")
pp_processor = PayPalProcessor()

processors = [cc_processor, pp_processor]
for processor in processors:
    print(processor.process_payment(100))

## 4. Real-World Complex Example: E-Commerce System 🛍️

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime
from typing import List, Dict, Any

class OrderStatus:
    PENDING = "PENDING"
    PROCESSING = "PROCESSING"
    SHIPPED = "SHIPPED"
    DELIVERED = "DELIVERED"
    CANCELLED = "CANCELLED"

class InventoryManager(ABC):
    @abstractmethod
    def check_availability(self, product_id: str, quantity: int) -> bool:
        pass
    
    @abstractmethod
    def reserve_items(self, product_id: str, quantity: int) -> bool:
        pass
    
    @abstractmethod
    def release_items(self, product_id: str, quantity: int) -> None:
        pass

class WarehouseInventory(InventoryManager):
    def __init__(self):
        self.__inventory: Dict[str, int] = {}
        self.__reserved: Dict[str, int] = {}
    
    def check_availability(self, product_id: str, quantity: int) -> bool:
        available = self.__inventory.get(product_id, 0) - \
                   self.__reserved.get(product_id, 0)
        return available >= quantity
    
    def reserve_items(self, product_id: str, quantity: int) -> bool:
        if self.check_availability(product_id, quantity):
            self.__reserved[product_id] = \
                self.__reserved.get(product_id, 0) + quantity
            return True
        return False
    
    def release_items(self, product_id: str, quantity: int) -> None:
        if product_id in self.__reserved:
            self.__reserved[product_id] -= quantity

class OrderProcessor(ABC):
    @abstractmethod
    def process_order(self, order: Dict[str, Any]) -> bool:
        pass
    
    @abstractmethod
    def cancel_order(self, order_id: str) -> bool:
        pass

class StandardOrderProcessor(OrderProcessor):
    def __init__(self, inventory_manager: InventoryManager):
        self.__inventory = inventory_manager
        self.__orders: Dict[str, Dict] = {}
    
    def process_order(self, order: Dict[str, Any]) -> bool:
        order_id = order['order_id']
        items = order['items']
        
        # Check and reserve inventory
        for item in items:
            if not self.__inventory.reserve_items(item['product_id'],
                                                 item['quantity']):
                # Rollback previous reservations
                self.__rollback_reservations(items)
                return False
        
        # Process payment
        if not self.__process_payment(order):
            self.__rollback_reservations(items)
            return False
        
        # Store order
        self.__orders[order_id] = {
            **order,
            'status': OrderStatus.PROCESSING,
            'timestamp': datetime.now()
        }
        
        return True
    
    def cancel_order(self, order_id: str) -> bool:
        if order_id not in self.__orders:
            return False
        
        order = self.__orders[order_id]
        if order['status'] not in [OrderStatus.PENDING, OrderStatus.PROCESSING]:
            return False
        
        # Release inventory
        for item in order['items']:
            self.__inventory.release_items(item['product_id'],
                                          item['quantity'])
        
        # Process refund
        self.__process_refund(order)
        
        order['status'] = OrderStatus.CANCELLED
        return True
    
    def __rollback_reservations(self, items: List[Dict]) -> None:
        for item in items:
            self.__inventory.release_items(item['product_id'],
                                          item['quantity'])
    
    def __process_payment(self, order: Dict) -> bool:
        # Complex payment processing logic
        return True
    
    def __process_refund(self, order: Dict) -> bool:
        # Complex refund processing logic
        return True

# Using the e-commerce system
inventory = WarehouseInventory()
order_processor = StandardOrderProcessor(inventory)

# Create an order
order = {
    'order_id': '12345',
    'customer_id': '67890',
    'items': [
        {'product_id': 'PROD1', 'quantity': 2},
        {'product_id': 'PROD2', 'quantity': 1}
    ],
    'payment': {
        'method': 'credit_card',
        'details': {
            'card_number': '**** **** **** 1234',
            'expiry': '12/24'
        }
    }
}

# Process the order
success = order_processor.process_order(order)
print(f"Order processing {'successful' if success else 'failed'}")

## 5. Advanced Abstraction: Event-Driven System 🎮

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List, Callable
from datetime import datetime
import json

class Event:
    def __init__(self, event_type: str, data: Dict):
        self.type = event_type
        self.data = data
        self.timestamp = datetime.now()
    
    def __str__(self):
        return f"Event({self.type}) at {self.timestamp}"

class EventListener(ABC):
    @abstractmethod
    def handle_event(self, event: Event) -> None:
        pass

class EventBus:
    def __init__(self):
        self.__listeners: Dict[str, List[EventListener]] = {}
        self.__event_history: List[Event] = []
    
    def subscribe(self, event_type: str, listener: EventListener) -> None:
        if event_type not in self.__listeners:
            self.__listeners[event_type] = []
        self.__listeners[event_type].append(listener)
    
    def publish(self, event: Event) -> None:
        self.__event_history.append(event)
        if event.type in self.__listeners:
            for listener in self.__listeners[event.type]:
                try:
                    listener.handle_event(event)
                except Exception as e:
                    self.__handle_error(e, event, listener)
    
    def __handle_error(self, error: Exception, event: Event,
                       listener: EventListener) -> None:
        error_event = Event(
            "error",
            {
                "original_event": event.type,
                "listener": listener.__class__.__name__,
                "error": str(error)
            }
        )
        self.publish(error_event)

class LoggingListener(EventListener):
    def handle_event(self, event: Event) -> None:
        print(f"[LOG] {event}")

class MetricsListener(EventListener):
    def __init__(self):
        self.__metrics: Dict[str, int] = {}
    
    def handle_event(self, event: Event) -> None:
        self.__metrics[event.type] = self.__metrics.get(event.type, 0) + 1
    
    def get_metrics(self) -> Dict[str, int]:
        return self.__metrics.copy()

class NotificationListener(EventListener):
    def handle_event(self, event: Event) -> None:
        if event.type == "error":
            self.__send_notification(f"Error occurred: {event.data}")
    
    def __send_notification(self, message: str) -> None:
        # Complex notification logic (email, SMS, etc.)
        print(f"[NOTIFICATION] {message}")

# Using the event system
event_bus = EventBus()

# Add listeners
logger = LoggingListener()
metrics = MetricsListener()
notifier = NotificationListener()

event_bus.subscribe("user_login", logger)
event_bus.subscribe("user_login", metrics)
event_bus.subscribe("error", notifier)

# Publish events
login_event = Event("user_login", {"user_id": "123", "ip": "192.168.1.1"})
event_bus.publish(login_event)

# Check metrics
print("\nMetrics:", metrics.get_metrics())

## 6. Enterprise-Level Example: Microservice Architecture 🏢

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List, Optional
from datetime import datetime
import json

class ServiceDiscovery(ABC):
    @abstractmethod
    def register_service(self, service_name: str, endpoint: str) -> None:
        pass
    
    @abstractmethod
    def get_service_endpoint(self, service_name: str) -> Optional[str]:
        pass

class CircuitBreaker:
    def __init__(self, failure_threshold: int = 5,
                 reset_timeout: int = 60):
        self.__failures = 0
        self.__last_failure_time = None
        self.__failure_threshold = failure_threshold
        self.__reset_timeout = reset_timeout
    
    def record_failure(self) -> None:
        self.__failures += 1
        self.__last_failure_time = datetime.now()
    
    def record_success(self) -> None:
        self.__failures = 0
        self.__last_failure_time = None
    
    def is_open(self) -> bool:
        if self.__failures >= self.__failure_threshold:
            if self.__last_failure_time:
                elapsed = (datetime.now() - 
                          self.__last_failure_time).total_seconds()
                if elapsed >= self.__reset_timeout:
                    self.__failures = 0
                    return False
            return True
        return False

class MessageBroker(ABC):
    @abstractmethod
    def publish(self, topic: str, message: Dict) -> None:
        pass
    
    @abstractmethod
    def subscribe(self, topic: str, callback: Callable) -> None:
        pass

class Microservice(ABC):
    def __init__(self, name: str, service_discovery: ServiceDiscovery,
                 message_broker: MessageBroker):
        self.name = name
        self.__service_discovery = service_discovery
        self.__message_broker = message_broker
        self.__circuit_breakers: Dict[str, CircuitBreaker] = {}
    
    def start(self) -> None:
        self.__register_service()
        self._setup_subscriptions()
        print(f"Service {self.name} started")
    
    @abstractmethod
    def _setup_subscriptions(self) -> None:
        pass
    
    def call_service(self, service_name: str,
                     operation: Callable) -> Optional[Any]:
        if service_name not in self.__circuit_breakers:
            self.__circuit_breakers[service_name] = CircuitBreaker()
        
        breaker = self.__circuit_breakers[service_name]
        if breaker.is_open():
            print(f"Circuit breaker open for {service_name}")
            return None
        
        try:
            result = operation()
            breaker.record_success()
            return result
        except Exception as e:
            breaker.record_failure()
            print(f"Error calling {service_name}: {e}")
            return None
    
    def __register_service(self) -> None:
        self.__service_discovery.register_service(
            self.name,
            f"http://localhost:8080/{self.name}"
        )

class OrderService(Microservice):
    def __init__(self, service_discovery: ServiceDiscovery,
                 message_broker: MessageBroker):
        super().__init__("order_service", service_discovery,
                         message_broker)
        self.__orders: Dict[str, Dict] = {}
    
    def _setup_subscriptions(self) -> None:
        self._message_broker.subscribe(
            "order.created",
            self.__handle_order_created
        )
    
    def create_order(self, order_data: Dict) -> str:
        order_id = str(len(self.__orders) + 1)
        self.__orders[order_id] = {
            **order_data,
            'status': 'created',
            'timestamp': datetime.now()
        }
        
        self._message_broker.publish(
            "order.created",
            {
                'order_id': order_id,
                'data': order_data
            }
        )
        
        return order_id
    
    def __handle_order_created(self, message: Dict) -> None:
        order_id = message['order_id']
        print(f"Processing order {order_id}")

# Example implementation of service discovery
class SimpleServiceDiscovery(ServiceDiscovery):
    def __init__(self):
        self.__services: Dict[str, str] = {}
    
    def register_service(self, service_name: str, endpoint: str) -> None:
        self.__services[service_name] = endpoint
    
    def get_service_endpoint(self, service_name: str) -> Optional[str]:
        return self.__services.get(service_name)

# Example implementation of message broker
class SimpleMessageBroker(MessageBroker):
    def __init__(self):
        self.__subscribers: Dict[str, List[Callable]] = {}
    
    def publish(self, topic: str, message: Dict) -> None:
        if topic in self.__subscribers:
            for callback in self.__subscribers[topic]:
                callback(message)
    
    def subscribe(self, topic: str, callback: Callable) -> None:
        if topic not in self.__subscribers:
            self.__subscribers[topic] = []
        self.__subscribers[topic].append(callback)

# Using the microservice architecture
service_discovery = SimpleServiceDiscovery()
message_broker = SimpleMessageBroker()

# Create and start order service
order_service = OrderService(service_discovery, message_broker)
order_service.start()

# Create an order
order_id = order_service.create_order({
    'customer_id': '123',
    'items': [
        {'product_id': 'ABC', 'quantity': 2},
        {'product_id': 'XYZ', 'quantity': 1}
    ]
})

## 7. Practice Exercises 🎯

### Exercise 1: Game Engine Abstraction
Create a game engine with abstract components for rendering, physics, and audio.

In [ ]:
from abc import ABC, abstractmethod
from typing import Tuple, List

class GameObject(ABC):
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
    
    @abstractmethod
    def update(self, delta_time: float) -> None:
        pass
    
    @abstractmethod
    def render(self) -> None:
        pass

class PhysicsComponent(ABC):
    @abstractmethod
    def apply_force(self, fx: float, fy: float) -> None:
        pass
    
    @abstractmethod
    def check_collision(self, other: 'PhysicsComponent') -> bool:
        pass

class AudioComponent(ABC):
    @abstractmethod
    def play_sound(self, sound_id: str) -> None:
        pass
    
    @abstractmethod
    def stop_sound(self, sound_id: str) -> None:
        pass

class Player(GameObject):
    def __init__(self, x: float, y: float):
        super().__init__(x, y)
        self.velocity = (0.0, 0.0)
        self.physics = PlayerPhysics()
        self.audio = PlayerAudio()
    
    def update(self, delta_time: float) -> None:
        self.x += self.velocity[0] * delta_time
        self.y += self.velocity[1] * delta_time
    
    def render(self) -> None:
        print(f"Rendering player at ({self.x}, {self.y})")

class PlayerPhysics(PhysicsComponent):
    def apply_force(self, fx: float, fy: float) -> None:
        # Implement physics calculations
        pass
    
    def check_collision(self, other: PhysicsComponent) -> bool:
        # Implement collision detection
        return False

class PlayerAudio(AudioComponent):
    def play_sound(self, sound_id: str) -> None:
        print(f"Playing sound: {sound_id}")
    
    def stop_sound(self, sound_id: str) -> None:
        print(f"Stopping sound: {sound_id}")

# Example usage
player = Player(100, 100)
player.update(0.16)  # Update with 16ms frame time
player.render()
player.audio.play_sound("jump")

### Exercise 2: Database Abstraction Layer
Create an abstract database interface that can work with different database types.

In [ ]:
from abc import ABC, abstractmethod
from typing import Any, List, Dict, Optional

class DatabaseConnection(ABC):
    @abstractmethod
    def connect(self) -> bool:
        pass
    
    @abstractmethod
    def disconnect(self) -> None:
        pass
    
    @abstractmethod
    def execute_query(self, query: str, params: Optional[tuple] = None) -> Any:
        pass

class QueryBuilder:
    def __init__(self):
        self.__select_fields: List[str] = []
        self.__from_table: str = ""
        self.__where_conditions: List[str] = []
        self.__order_by: List[str] = []
        self.__limit: Optional[int] = None
    
    def select(self, *fields: str) -> 'QueryBuilder':
        self.__select_fields.extend(fields)
        return self
    
    def from_table(self, table: str) -> 'QueryBuilder':
        self.__from_table = table
        return self
    
    def where(self, condition: str) -> 'QueryBuilder':
        self.__where_conditions.append(condition)
        return self
    
    def order_by(self, field: str, desc: bool = False) -> 'QueryBuilder':
        direction = "DESC" if desc else "ASC"
        self.__order_by.append(f"{field} {direction}")
        return self
    
    def limit(self, limit: int) -> 'QueryBuilder':
        self.__limit = limit
        return self
    
    def build(self) -> str:
        query_parts = []
        
        # SELECT
        fields = ", ".join(self.__select_fields) or "*"
        query_parts.append(f"SELECT {fields}")
        
        # FROM
        query_parts.append(f"FROM {self.__from_table}")
        
        # WHERE
        if self.__where_conditions:
            conditions = " AND ".join(self.__where_conditions)
            query_parts.append(f"WHERE {conditions}")
        
        # ORDER BY
        if self.__order_by:
            order = ", ".join(self.__order_by)
            query_parts.append(f"ORDER BY {order}")
        
        # LIMIT
        if self.__limit is not None:
            query_parts.append(f"LIMIT {self.__limit}")
        
        return " ".join(query_parts)

class SQLiteConnection(DatabaseConnection):
    def __init__(self, db_path: str):
        self.__db_path = db_path
        self.__connection = None
    
    def connect(self) -> bool:
        print(f"Connecting to SQLite database at {self.__db_path}")
        return True
    
    def disconnect(self) -> None:
        print("Disconnecting from SQLite database")
    
    def execute_query(self, query: str,
                      params: Optional[tuple] = None) -> Any:
        print(f"Executing SQLite query: {query}")
        if params:
            print(f"With parameters: {params}")
        return []

class PostgreSQLConnection(DatabaseConnection):
    def __init__(self, host: str, port: int, database: str,
                 user: str, password: str):
        self.__host = host
        self.__port = port
        self.__database = database
        self.__user = user
        self.__password = password
        self.__connection = None
    
    def connect(self) -> bool:
        print(f"Connecting to PostgreSQL database at {self.__host}:{self.__port}")
        return True
    
    def disconnect(self) -> None:
        print("Disconnecting from PostgreSQL database")
    
    def execute_query(self, query: str,
                      params: Optional[tuple] = None) -> Any:
        print(f"Executing PostgreSQL query: {query}")
        if params:
            print(f"With parameters: {params}")
        return []

# Example usage
# Create query
query = QueryBuilder()\
    .select("id", "name", "email")\
    .from_table("users")\
    .where("age >= 18")\
    .order_by("name")\
    .limit(10)\
    .build()

# Use with SQLite
sqlite_db = SQLiteConnection("users.db")
sqlite_db.connect()
sqlite_db.execute_query(query)
sqlite_db.disconnect()

# Use with PostgreSQL
postgres_db = PostgreSQLConnection(
    host="localhost",
    port=5432,
    database="users",
    user="admin",
    password="password"
)
postgres_db.connect()
postgres_db.execute_query(query)
postgres_db.disconnect()

### Exercise 3: File System Abstraction
Create an abstract file system interface that can work with different storage types.

In [ ]:
from abc import ABC, abstractmethod
from typing import List, BinaryIO, Optional
from datetime import datetime

class FileInfo:
    def __init__(self, name: str, size: int,
                 created: datetime, modified: datetime):
        self.name = name
        self.size = size
        self.created = created
        self.modified = modified
    
    def __str__(self) -> str:
        return f"{self.name} ({self.size} bytes)"

class FileSystem(ABC):
    @abstractmethod
    def read_file(self, path: str) -> bytes:
        pass
    
    @abstractmethod
    def write_file(self, path: str, data: bytes) -> None:
        pass
    
    @abstractmethod
    def delete_file(self, path: str) -> None:
        pass
    
    @abstractmethod
    def list_directory(self, path: str) -> List[FileInfo]:
        pass
    
    @abstractmethod
    def create_directory(self, path: str) -> None:
        pass
    
    @abstractmethod
    def delete_directory(self, path: str) -> None:
        pass

class LocalFileSystem(FileSystem):
    def read_file(self, path: str) -> bytes:
        print(f"Reading local file: {path}")
        return b""
    
    def write_file(self, path: str, data: bytes) -> None:
        print(f"Writing {len(data)} bytes to local file: {path}")
    
    def delete_file(self, path: str) -> None:
        print(f"Deleting local file: {path}")
    
    def list_directory(self, path: str) -> List[FileInfo]:
        print(f"Listing local directory: {path}")
        return []
    
    def create_directory(self, path: str) -> None:
        print(f"Creating local directory: {path}")
    
    def delete_directory(self, path: str) -> None:
        print(f"Deleting local directory: {path}")

class S3FileSystem(FileSystem):
    def __init__(self, bucket: str, access_key: str, secret_key: str):
        self.__bucket = bucket
        self.__access_key = access_key
        self.__secret_key = secret_key
    
    def read_file(self, path: str) -> bytes:
        print(f"Reading from S3: {self.__bucket}/{path}")
        return b""
    
    def write_file(self, path: str, data: bytes) -> None:
        print(f"Writing to S3: {self.__bucket}/{path}")
    
    def delete_file(self, path: str) -> None:
        print(f"Deleting from S3: {self.__bucket}/{path}")
    
    def list_directory(self, path: str) -> List[FileInfo]:
        print(f"Listing S3 directory: {self.__bucket}/{path}")
        return []
    
    def create_directory(self, path: str) -> None:
        # S3 doesn't need directory creation
        pass
    
    def delete_directory(self, path: str) -> None:
        print(f"Deleting S3 directory: {self.__bucket}/{path}")

# Example usage
local_fs = LocalFileSystem()
s3_fs = S3FileSystem("my-bucket", "access-key", "secret-key")

# Use local file system
local_fs.create_directory("/tmp/test")
local_fs.write_file("/tmp/test/file.txt", b"Hello, World!")
local_fs.list_directory("/tmp/test")

# Use S3 file system
s3_fs.write_file("test/file.txt", b"Hello, World!")
s3_fs.list_directory("test")

## 8. Advanced Concepts: Dependency Injection 🔄

Implement a dependency injection container with abstract factories.

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Type, Any, Callable

class Container:
    def __init__(self):
        self.__bindings: Dict[Type, Callable[[], Any]] = {}
        self.__singletons: Dict[Type, Any] = {}
    
    def bind(self, interface: Type,
             implementation: Callable[[], Any]) -> None:
        self.__bindings[interface] = implementation
    
    def singleton(self, interface: Type,
                  implementation: Callable[[], Any]) -> None:
        def singleton_factory() -> Any:
            if interface not in self.__singletons:
                self.__singletons[interface] = implementation()
            return self.__singletons[interface]
        self.__bindings[interface] = singleton_factory
    
    def resolve(self, interface: Type) -> Any:
        if interface not in self.__bindings:
            raise ValueError(f"No binding for {interface}")
        return self.__bindings[interface]()

# Example usage with previous database abstraction
class Database(ABC):
    @abstractmethod
    def query(self, sql: str) -> List[Dict]:
        pass

class MySQLDatabase(Database):
    def query(self, sql: str) -> List[Dict]:
        print(f"Executing MySQL query: {sql}")
        return []

class UserRepository:
    def __init__(self, db: Database):
        self.__db = db
    
    def find_by_id(self, user_id: int) -> Optional[Dict]:
        results = self.__db.query(f"SELECT * FROM users WHERE id = {user_id}")
        return results[0] if results else None

# Set up dependency injection
container = Container()
container.singleton(Database, lambda: MySQLDatabase())

# Use the container
db = container.resolve(Database)
user_repo = UserRepository(db)
user = user_repo.find_by_id(1)

## 9. Best Practices 📚

1. Keep abstractions focused and cohesive
2. Follow the Interface Segregation Principle
3. Use dependency injection for better testability
4. Document your abstractions clearly
5. Don't over-abstract - keep it practical
6. Consider using abstract base classes for interface definitions
7. Use type hints for better code clarity

## 10. Summary 🎉

- Abstraction helps manage complexity
- Abstract base classes define interfaces
- Dependency injection promotes loose coupling
- Real-world systems benefit from proper abstraction
- Python provides powerful tools for abstraction

Keep practicing these concepts to build maintainable and scalable systems!
